In [6]:
import pandas as pd
import random
import os
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.multioutput import MultiOutputRegressor
from sklearn import metrics

In [7]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(42) # Seed 고정

In [3]:
train_df = pd.read_csv('./train.csv')

In [4]:
train_x = train_df.filter(regex='X') # Input : X Featrue
train_y = train_df.filter(regex='Y') # Output : Y Feature

In [5]:
LR = MultiOutputRegressor(LinearRegression()).fit(train_x, train_y)
print('Done.')

Done.


In [6]:
test_x = pd.read_csv('./test.csv').drop(columns=['ID'])

In [7]:
preds = LR.predict(test_x)
print('Done.')

Done.


In [8]:
submit = pd.read_csv('./sample_submission.csv')

In [9]:
for idx, col in enumerate(submit.columns):
    if col=='ID':
        continue
    submit[col] = preds[:,idx-1]
print('Done.')

Done.


In [10]:
# submit.to_csv('./submit.csv', index=False)

In [8]:
def lg_nrmse(gt, preds):
    # 각 Y Feature별 NRMSE 총합
    # Y_01 ~ Y_08 까지 20% 가중치 부여
    all_nrmse = []
    for idx in range(14): # ignore 'ID'
        rmse = metrics.mean_squared_error(gt[:,idx], preds[:,idx], squared=False)
        nrmse = rmse/np.mean(np.abs(gt[:,idx]))
        all_nrmse.append(nrmse)
    score = 1.2 * np.sum(all_nrmse[:8]) + 1.0 * np.sum(all_nrmse[8:15])
    return score

In [14]:
df = pd.read_csv('./train.csv')

train_df, valid_df = train_test_split(df, train_size=0.8)

train_x = train_df.filter(regex='X') # Input : X Featrue
train_y = train_df.filter(regex='Y') # Output : Y Feature

valid_x = valid_df.filter(regex='X') # Input : X Featrue
valid_y = valid_df.filter(regex='Y') # Output : Y Feature

test_x = pd.read_csv('./test.csv').drop(columns=['ID'])

LR = MultiOutputRegressor(LinearRegression()).fit(train_x, train_y)

valid_preds = LR.predict(valid_x)
lg_nrmse(valid_y.values, valid_preds)

1.9929924217815767

In [10]:
test_x = pd.read_csv('./test.csv').drop(columns=['ID'])
preds = LR.predict(test_x)
submit = pd.read_csv('./sample_submission.csv')
for idx, col in enumerate(submit.columns):
    if col=='ID':
        continue
    submit[col] = preds[:,idx-1]
submit.to_csv('./range_preprocessing.csv', index=False)

In [13]:
pd.DataFrame(preds).describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
count,39608.000000,39608.000000,39608.000000,39608.000000,39608.000000,39608.000000,39608.000000,39608.000000,39608.000000,39608.000000,39608.000000,39608.000000,39608.000000,39608.000000
mean,1.333525,1.034667,0.989047,13.718584,31.348808,16.644527,3.148411,-26.269159,-26.282657,-22.352389,24.358498,-26.211964,-26.208323,-26.221036
std,0.058804,0.067158,0.055265,0.459746,0.372132,0.095647,0.057401,0.157361,0.149712,0.232010,0.103189,0.151055,0.151686,0.150458
min,0.722527,0.462723,0.537312,8.763135,28.771916,15.738457,2.769825,-27.851366,-27.859793,-23.910278,23.357127,-27.823444,-27.816108,-27.800743
25%,1.294776,0.990416,0.951360,13.425834,31.109467,16.581900,3.111318,-26.392911,-26.397400,-22.507229,24.290653,-26.328105,-26.325622,-26.335967
50%,1.332658,1.038451,0.987433,13.687228,31.354103,16.643817,3.148922,-26.257997,-26.274840,-22.320020,24.356561,-26.202287,-26.198435,-26.212280
75%,1.371831,1.081475,1.025536,13.974681,31.590275,16.706202,3.186205,-26.149058,-26.171849,-22.186978,24.423807,-26.099286,-26.094873,-26.109709
max,1.622975,1.333523,1.433023,19.771395,34.629456,18.216004,3.622615,-24.818176,-24.823844,-21.227604,25.750111,-24.745805,-24.755745,-24.778878
